In [ ]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.runBlocking

val username = "Johann-21.0"
val intervalStart = LocalDate.parse("2019-01-01")
val intervalEnd = LocalDate.parse("2020-09-30")
val granularity = TimeGranularity.MONTHLY
val fundName = "Expenses"
val reportViewName = "Expenses report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

val client = FundsClient()

In [ ]:
val user = runBlocking { client.ensureUserExists(username) }
user

In [ ]:
val reportView = runBlocking {
    client.createReportView(user, reportViewName, fundName, File(REPORT_DATA_CONFIGURATION_YAML_FILE))
}
reportView

In [ ]:
val reportData = runBlocking {
    client.getReportViewData(user, reportViewName, intervalStart, intervalEnd, granularity)
}
reportData

In [ ]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun plotGroupData(plotTitle: String, groupItemFilter: (ReportDataGroupItemTO) -> Boolean): Plot =
    dataFrameOf(
        "month" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to reportData.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.net!!.negate() }.reduce { acc, value -> acc + value } },
        "allocated" to reportData.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value } },
        "left" to reportData.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value } },
    )
        .plot {
            x("month")
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.YELLOW
                }
            }
            layout {
                title = plotTitle
                size = 1200 to 600
            }
        }

fun plotGroup(title: String, group: String): Plot =
    plotGroupData(title) { it.group == group }

fun plotGroupsTotal(): Plot =
    plotGroupData("Total expenses") { true }



In [ ]:
plotGroupsTotal()

In [ ]:
plotGroup("Basic Expenses", "basic")

In [ ]:
plotGroup("Home Expenses", "home")

In [ ]:
plotGroup("Transport Expenses", "transport")

In [ ]:
plotGroup("Shopping & Services Expenses", "shopping_services")

In [ ]:
plotGroup("Fun Expenses", "fun")

In [ ]:
plotGroup("Gifts Expenses", "gifts")

In [ ]:
plotGroup("Development Expenses", "development")

In [ ]:
plotGroup("Investment Expenses", "investment")